# Tutorial Walkthrough
1. Install Encord Active 
2. Build a custom metric
3. Download a prebuilt project
4. Run the custom metric on the project

## Install Encord Active

In [ ]:
# enforce (install) python 3.9
!sudo apt-get -qq install python3.9 -q
!sudo apt -qq install python3.9-distutils -q
!sudo update-alternatives --install /usr/bin/python3 python3 /usr/bin/python3.9 10
!sudo apt -qq install -q python3-pip
!python -m pip install -q --upgrade pip
!python -m pip install -q --upgrade setuptools

# install encord-active
!python -m pip install encord-active

debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 76, <> line 4.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 
Selecting previously unselected package libpython3.9-minimal:amd64.
(Reading database ... 124013 files and directories currently installed.)
Preparing to unpack .../libpython3.9-minimal_3.9.16-1+bionic1_amd64.deb ...
Unpacking libpython3.9-minimal:amd64 (3.9.16-1+bionic1) ...
Selecting previously unselected package python3.9-minimal.
Preparing to unpack .../python3.9-minimal_3.9.16-1+bionic1_amd64.deb ...
Unpacking python3.9-minimal (3.9.16-1+bionic1) ...
Selecting previously unselected package libpython3.9-stdlib:amd64.
Preparing to unpack .../

## Build a custom metric
We will search for deviation of objects' instance occurrences (comparing centers)

E.g. An object instance of type 'truck' appears on frames 1, 2 and 3 of a video, so we are going to calculate its [geometric center](https://en.wikipedia.org/wiki/Centroid) for each frame, and use those centers to find their deviation from the median of the instance's centers.

In [ ]:
%%file metric.py
from collections import defaultdict

import numpy as np
from encord_active.lib.common import utils
from encord_active.lib.common.iterator import Iterator
from encord_active.lib.metrics.metric import AnnotationType, DataType, Metric, MetricType
from encord_active.lib.common.writer import CSVMetricWriter
from loguru import logger

logger = logger.opt(colors=True)


class InstanceDeviation(Metric):
    TITLE = "Instance Deviation"
    METRIC_TYPE = MetricType.HEURISTIC
    DATA_TYPE = DataType.IMAGE
    ANNOTATION_TYPE = [AnnotationType.OBJECT.BOUNDING_BOX, AnnotationType.OBJECT.POLYGON]
    SHORT_DESCRIPTION = "Find deviation of an object's instance comparing it to its other instances."
    LONG_DESCRIPTION = r"""Find deviation of an object's instance comparing it to its other instances.  
Deviation is calculated using center of the objects.
"""

    def test(self, iterator: Iterator, writer: CSVMetricWriter):
        valid_annotation_types = {annotation_type.value for annotation_type in self.ANNOTATION_TYPE}

        object_hash_to_centroids = defaultdict(dict)

        # Separate objects' instances (same objectHash [aka track id] means same object instance)
        for data_unit, img_pth in iterator.iterate(desc="Custom progress description"):
            frame_id = data_unit["data_sequence"]
            for obj in data_unit["labels"].get("objects", []):
                # Only analyse objects with valid shapes (bounding boxes and polygons)
                if not obj["shape"] in valid_annotation_types:
                    continue
                obj_hash = obj["objectHash"]
                # Check out 'utils.get_object_coordinates' to learn more about Encord's object coordinates extraction
                poly = utils.get_polygon(obj)
                if poly is not None:
                    obj_centroid = tuple(poly.centroid.coords)
                    object_hash_to_centroids[obj_hash][frame_id] = obj_centroid

        # Calculate deviation
        object_hash_to_centroid_mean = defaultdict(float)
        for obj_hash, centroids in object_hash_to_centroids.items():
            centroid_mean = sum(centroids.values(), np.array([0, 0])) / len(centroids)
            object_hash_to_centroid_mean[obj_hash] = centroid_mean

        # Evaluate deviation on each object instance and save it
        for data_unit, img_pth in iterator.iterate(desc="Custom progress description"):
            frame_id = data_unit["data_sequence"]
            for obj in data_unit["labels"].get("objects", []):
                # Only analyse objects with valid shapes (bounding boxes and polygons)
                if not obj["shape"] in valid_annotation_types:
                    continue
                obj_hash = obj["objectHash"]
                if frame_id in object_hash_to_centroids[obj_hash].keys():
                    obj_centroid = object_hash_to_centroids[obj_hash][frame_id]
                    obj_score = np.linalg.norm(obj_centroid - object_hash_to_centroid_mean[obj_hash])
                    writer.write(obj_score, labels=obj)

if __name__ == "__main__":
    import sys
    from pathlib import Path

    from encord_active.lib.common.tester import perform_test

    path = sys.argv[1]
    perform_test(InstanceDeviation(), data_dir=Path(path), use_cache_only=True)

Writing metric.py


## Download a prebuilt project

In [ ]:
project_name = "[open-source]-covid-19-segmentations"

!encord-active config set projects_dir '/content/.'
!encord-active download --project-name $project_name

2022-12-09 10:55:28.810 INFO    matplotlib.font_manager: generated new fontManager
Traceback (most recent call last):
  File "/usr/local/bin/encord-active", line 5, in <module>
    from encord_active.app.main import cli
  File "/usr/local/lib/python3.9/dist-packages/encord_active/app/main.py", line 18, in <module>
    from encord_active.lib.coco.importer import CocoImporter
  File "/usr/local/lib/python3.9/dist-packages/encord_active/lib/coco/importer.py", line 19, in <module>
    from encord_active.lib.coco.parsers import (
  File "/usr/local/lib/python3.9/dist-packages/encord_active/lib/coco/parsers.py", line 11, in <module>
    from encord_active.lib.coco.utils import annToMask
  File "/usr/local/lib/python3.9/dist-packages/encord_active/lib/coco/utils.py", line 6, in <module>
    from pycocotools.mask import decode, frPyObjects, merge
ModuleNotFoundError: No module named 'pycocotools'
Traceback (most recent call last):
  File "/usr/local/bin/encord-active", line 5, in <module>
    

## Run the custom metric on the project

In [ ]:
!python metric.py "/content/"$project_name

2022-12-09 10:55:33.283 | ERROR    | encord_active.lib.common.utils:fetch_project_info:21 - Couldn't find meta file for project
2022-12-09 10:55:33.283 | ERROR    | __main__:<module>:68 - An error has been caught in function '<module>', process 'MainProcess' (1055), thread 'MainThread' (140117598545792):
Traceback (most recent call last):

> File "/content/metric.py", line 68, in <module>
    perform_test(InstanceDeviation(), data_dir=Path(path), use_cache_only=True)
    │            │                             │    └ '/content/[open-source]-covid-19-segmentations'
    │            │                             └ <class 'pathlib.Path'>
    │            └ <class '__main__.InstanceDeviation'>
    └ <function perform_test at 0x7f6f8f35ea60>

  File "/usr/local/lib/python3.9/dist-packages/encord_active/lib/common/tester.py", line 42, in perform_test
    project = fetch_project_info(data_dir)
              │                  └ PosixPath('/content/[open-source]-covid-19-segmentations')
   